# Leitura da base da dados

### 1. Upside Capture

O Upside Capture Ratio é uma métrica que mede a capacidade de um fundo ou estratégia de capturar os movimentos positivos do mercado em relação ao seu benchmark. Um ratio acima de 100% indica que o fundo tende a superar o benchmark em períodos de alta, enquanto um valor abaixo sugere underperformance nesses cenários.

#### 1.1 Filtre os Períodos de Alta do Benchmark
Identifique os períodos em que o benchmark teve retornos positivos:
$$R_{bench} > 0$$

#### 1.2 Calcule o Upside Capture Ratio
A fórmula é:
    $$Upside\ Capture\ Ratio =    \frac{média\ dos\ retornos\ do\ fundo\ nos\ periodos\ de\ alta\ do\ benchmar}{média\ dos\ retornos\ de\ alta\ do\ benchmark}$$

#### 1.2 Interpretação
 `> 100%`: Seu fundo captura mais ganhos que o benchmark em rallies.

`= 100%`: Performance igual ao benchmark em alta.

`< 100%`: Seu fundo sobe menos que o benchmark em alta.



#### 2. Implementação do Downside Capture Ratio

O Downside Capture Ratio mede o desempenho do seu fundo em relação ao benchmark durante períodos negativos. Um valor abaixo de 100% é desejável, pois indica que seu fundo perde menos que o benchmark em quedas.

####  2.1 Filtre os Períodos de Baixa do Benchmark
Identifique os períodos em que o benchmark teve retornos negativos:
$$R_{bench} < 0$$

#### Interpretação
`< 100%` Fundo cai menos que o benchmark (ideal)

 `= 100%` Comportamento igual ao benchmark

 `> 100%` Fundo cai mais que o benchmark.


#### 3. Capture Ratio (Relação de Captura)
A forma mais comum de combinar os dois indicadores:
Indice de Captura  = Upside Capture Ratio / Downside Capture Ratio

 
#### 3.1 Interpretação:

`> 1`: O fundo captura mais ganhos em altas do que perdas em baixas (ideal)

`= 1`: Comportamento simétrico

`< 1`: Sofre mais nas quedas do que ganha nas altas;


#### 4. Capture Spread (Diferencial de Captura)
Indicador simples de diferença:

Spread = Upside Capture − Downside Capture

#### 4.1 Interpretação:

`Positivo`: Bom (ganha mais do que perde)

`Negativo`: Ruim (perde mais do que ganha)


#### 5 Capture Efficiency (Eficiência de Captura)

Eficiencia = Upside Capture - 100%/ Downside Capture - 100% 
Interpretação:

`> 1`: Cada 1% de risco adicional traz mais de 1% de retorno extra

`< 1`: Risco não compensado por retorno

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate

In [32]:
dir = r'C:\Users\dieggo.araujo\Documents\Analise_FIA_Capture_Rateio\dados_raw\Retorno dos Fatores 2023 a 2024.xlsx'

In [40]:
# Função para calcular Upside e Downside Capture Ratios
def calculate_capture_ratios(fund_returns, bench_returns):
    # Upside Capture
    up_mask = bench_returns > 0
    upside_fund = fund_returns[up_mask]
    upside_bench = bench_returns[up_mask]
    upside_ratio = (np.mean(upside_fund) / np.mean(upside_bench)) * 100 if len(upside_bench) > 0 else np.nan
    
    # Downside Capture
    down_mask = bench_returns < 0
    downside_fund = fund_returns[down_mask]
    downside_bench = bench_returns[down_mask]
    downside_ratio = (np.mean(downside_fund) / np.mean(downside_bench)) * 100 if len(downside_bench) > 0 else np.nan
    
    return upside_ratio, downside_ratio

# Lista de fatores e dicionário de saída
fatores = ['Defensive', 'Valor', 'Momentum', 'Growth', 'SB']
bench = 'IBOV'
fatores_dict = {}

# Supondo que cada planilha tem colunas 'Retorno_Fundo' e 'Retorno_Benchmark'
for fator in fatores:
    df = pd.read_excel(dir, sheet_name=fator)
    
    # Calcula os ratios (ajuste os nomes das colunas conforme seu arquivo)
    upside, downside = calculate_capture_ratios(
        df[fator].values,
        df[bench].values
    )
    
    # Armazena no dicionário
    fatores_dict[fator] = {
        'dataframe': df,  # Opcional: manter o DataFrame original
        'upside_ratio': upside,
        'downside_ratio': downside,
        'capture_ratio': upside / downside if downside != 0 else np.nan,  # Índice combinado
        'spread': upside - downside
    }
    


table_data = []
for fator, metrics in fatores_dict.items():
    table_data.append([
        fator,
        f"{metrics['upside_ratio']:.2f}%",
        f"{metrics['downside_ratio']:.2f}%",
        f"{metrics['capture_ratio']:.2f}",
        f'{metrics['spread']:.2f}'
    ])

headers = ["Fator", "Upside", "Downside", "Capture Ratio", "spread"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+-----------+----------+------------+-----------------+----------+
| Fator     | Upside   | Downside   |   Capture Ratio |   spread |
+===========+==========+============+=================+==========+
| Defensive | 87.98%   | 79.84%     |            1.1  |     8.14 |
+-----------+----------+------------+-----------------+----------+
| Valor     | 101.92%  | 96.29%     |            1.06 |     5.63 |
+-----------+----------+------------+-----------------+----------+
| Momentum  | 97.03%   | 89.45%     |            1.08 |     7.58 |
+-----------+----------+------------+-----------------+----------+
| Growth    | 102.00%  | 93.66%     |            1.09 |     8.34 |
+-----------+----------+------------+-----------------+----------+
| SB        | 97.23%   | 87.53%     |            1.11 |     9.7  |
+-----------+----------+------------+-----------------+----------+


In [1]:
import numpy as np

def capture_indicators(returns_fund, returns_bench):
    # Upside Capture
    up_mask = returns_bench > 0
    fund_up = returns_fund[up_mask]
    bench_up = returns_bench[up_mask]
    upside = (np.mean(fund_up) / np.mean(bench_up)) * 100 if len(bench_up) > 0 else np.nan
    
    # Downside Capture
    down_mask = returns_bench < 0
    fund_down = returns_fund[down_mask]
    bench_down = returns_bench[down_mask]
    downside = (np.mean(fund_down) / np.mean(bench_down)) * 100 if len(bench_down) > 0 else np.nan
    
    # Combined Indicators
    capture_ratio = upside / downside
    capture_spread = upside - downside
    capture_efficiency = (upside - 100) / (downside - 100) if downside != 100 else np.nan
    
    return {
        "upside": upside,
        "downside": downside,
        "capture_ratio": capture_ratio,
        "capture_spread": capture_spread,
        "capture_efficiency": capture_efficiency
    }

# Exemplo de uso
returns_fund = np.array([0.03, -0.02, 0.05, -0.01, 0.04, -0.03])
returns_bench = np.array([0.02, -0.03, 0.04, -0.02, 0.03, -0.04])

results = capture_indicators(returns_fund, returns_bench)

print(f"Upside Capture: {results['upside']:.2f}%")
print(f"Downside Capture: {results['downside']:.2f}%")
print(f"Capture Ratio: {results['capture_ratio']:.2f}")
print(f"Capture Spread: {results['capture_spread']:.2f}%")
print(f"Capture Efficiency: {results['capture_efficiency']:.2f}")

Upside Capture: 133.33%
Downside Capture: 66.67%
Capture Ratio: 2.00
Capture Spread: 66.67%
Capture Efficiency: -1.00


In [4]:
import matplotlib.pyplot as plt

# Dados
labels = ['Upside', 'Downside', 'Ratio', 'Spread']
values = [results['upside'], results['downside'], 
          results['capture_ratio'], results['capture_spread']]

# Cria dashboard
fig, ax = plt.subplots(2, 2, figsize=(12, 10))

# Upside-Downside
ax[0,0].bar(['Upside', 'Downside'], [results['upside'], results['downside']], 
            color=['green', 'red'])
ax[0,0].set_title('Captura de Mercado')

# Capture Ratio
ax[0,1].axhline(1, color='gray', linestyle='--')
ax[0,1].bar('Ratio', results['capture_ratio'], 
            color='gold' if results['capture_ratio'] > 1 else 'silver')
ax[0,1].set_title('Capture Ratio')

# Spread
ax[1,0].axhline(0, color='gray', linestyle='--')
ax[1,0].bar('Spread', results['capture_spread'], 
            color='green' if results['capture_spread'] > 0 else 'red')
ax[1,0].set_title('Capture Spread')

# Eficiência
ax[1,1].axhline(1, color='gray', linestyle='--')
ax[1,1].bar('Efficiency', results['capture_efficiency'], 
            color='blue' if results['capture_efficiency'] > 1 else 'orange')
ax[1,1].set_title('Eficiência de Captura')

plt.tight_layout()
plt.show()

ImportError: DLL load failed while importing _path: Não foi possível encontrar o módulo especificado.